In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
x_train.mean(),x_train.std()

(tensor(0.1304), tensor(0.3073))

In [5]:
train_mean,train_std = x_train.mean(),x_train.std()

In [6]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean,train_std = x_train.mean(),x_train.std()

In [8]:
assert x_train.mean().abs()<1e-3
assert (1-x_train.std()).abs()<1e-3

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [16]:
# num hidden
nh = 50

In [17]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(m)
b2 = torch.zeros(1)

In [18]:
assert w1.mean().abs()<1e-3
assert (w1.std()-1/math.sqrt(m)).abs()<1e-3

In [19]:
def relu(x): return x.clamp_min(0.)
def lin(x, w, b): return x@w + b

In [20]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0058), tensor(0.9924))

In [21]:
t = lin(x_valid, w1, b1)

In [22]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0690), tensor(1.0274))

In [23]:
t = relu(lin(x_valid, w1, b1))

In [24]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4390), tensor(0.6285))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [25]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [26]:
w1.mean(),w1.std()

(tensor(0.0004), tensor(0.0505))

In [27]:
from torch.nn import init

In [28]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [29]:
w1.mean(),w1.std()

(tensor(5.7699e-05), tensor(0.0507))

In [30]:
t.mean(),t.std()

(tensor(0.5049), tensor(0.7638))

In [31]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [32]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1259), tensor(0.9072))

In [33]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [34]:
%timeit -n 10 _=model(x_valid)

8.1 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

In [41]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [42]:
def mse(output, targ): return (output.squeeze() - targ).pow(2).mean()

In [43]:
y_train,y_valid = y_train.float(),y_valid.float()

In [44]:
preds = model(x_train)

In [45]:
preds.shape

torch.Size([50000, 1])

In [46]:
mse(preds, y_train)

tensor(30.3267)

### Gradients and backward pass

In [90]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [91]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [92]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [119]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)
    return out,l1,l2

In [120]:
out,l1,l2 = forward_and_backward(x_train, y_train)

In [96]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
og  = x_train.g.clone()

In [128]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)

In [129]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b1
    l2 = relu(l1)
    out = l2 @ w22 + b2
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [130]:
loss = forward(xt2, y_train)

In [131]:
loss.backward()

In [133]:
assert torch.allclose(w12.grad, w1g, rtol=rtol, atol=atol)

## Refactor model

### Layers as classes

In [97]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [98]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [99]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [100]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [101]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [102]:
model = Model(w1, b1, w2, b2)

In [103]:
%time loss = model(x_train, y_train)

CPU times: user 86.5 ms, sys: 4.87 ms, total: 91.4 ms
Wall time: 45.3 ms


In [104]:
%time model.backward()

CPU times: user 2.95 s, sys: 3.83 s, total: 6.78 s
Wall time: 3.43 s


In [ ]:
rtol = 1e-3
atol = 1e-4
assert torch.allclose(w1g,w1.g, rtol=rtol, atol=atol)

In [115]:
assert torch.allclose(w2g,w2.g, rtol=rtol, atol=atol)

AssertionError: 

In [116]:
assert torch.allclose(b1g,b1.g, rtol=rtol, atol=atol)

AssertionError: 

In [117]:
assert torch.allclose(b2g,b2.g, rtol=rtol, atol=atol)

AssertionError: 

In [118]:
assert torch.allclose(og,x_train.g, rtol=rtol, atol=atol)

### Module.forward()

In [60]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [61]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [62]:
class Lin(Module):
    def __init__(self, ni, nf):
        self.w = torch.randn(ni,nf)/math.sqrt(ni)
        self.b = torch.zeros(nf)
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [63]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [85]:
class Model():
    def __init__(self, n_in, nh, n_out):
        self.layers = [Lin(n_in,nh), Relu(), Lin(nh,n_out)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [65]:
model = Model(m, nh, 1)

In [66]:
%time loss = model(x_train, y_train)

CPU times: user 90.4 ms, sys: 0 ns, total: 90.4 ms
Wall time: 45.5 ms


In [67]:
%time model.backward()

CPU times: user 218 ms, sys: 64.2 ms, total: 282 ms
Wall time: 141 ms


### Without einsum

In [86]:
class Lin(Module):
    def __init__(self, ni, nf):
        self.w = torch.randn(ni,nf)/math.sqrt(ni)
        self.b = torch.zeros(nf)
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        set_trace()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [87]:
model = Model(m, nh, 1)

In [88]:
%time loss = model(x_train, y_train)

CPU times: user 140 ms, sys: 1.07 ms, total: 141 ms
Wall time: 70.4 ms


In [89]:
%time model.backward()

> <ipython-input-86-3d32f044b888>(11)bwd()
      8     def bwd(self, out, inp):
      9         inp.g = out.g @ self.w.t()
     10         set_trace()
---> 11         self.w.g = inp.t() @ out.g
     12         self.b.g = out.g.sum(0)

ipdb> q


BdbQuit: 

### nn.Linear and nn.Module

In [74]:
#export
from torch import nn

In [75]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [82]:
model = Model(m, nh, 1)

In [83]:
%time loss = model(x_train, y_train)

CPU times: user 94.3 ms, sys: 0 ns, total: 94.3 ms
Wall time: 46.9 ms


In [84]:
%time loss.backward()

CPU times: user 108 ms, sys: 3.24 ms, total: 111 ms
Wall time: 55.5 ms


## Export

In [ ]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
